In [188]:
import pandas as pd 
import folium
import requests
import warnings
warnings.filterwarnings('ignore')

# Utility functions

In [189]:
def preprocess_data_per_year(data,df_department,year):
    # Get data relative to the year entered
    df_elec=data.loc[data["Année"]==year]
    # Remove dupplicate values for columns ...
    df_elec.drop_duplicates(subset=['Code INSEE de la commune'],inplace=True)
    # Drop missing values 
    df_elec.dropna(subset=['Code INSEE de la commune', 'Consommation annuelle moyenne de la commune (MWh)', 'Nom de la commune'],inplace=True)
    df_elec.rename(columns={'Code INSEE de la commune': 'code_commune_INSEE'}, inplace=True)
    df_elec['code_commune_INSEE']=df_elec['code_commune_INSEE'].astype(str)
    INSEE_codes=set(df_elec['code_commune_INSEE'])
    df_department=df_department.loc[df_department['code_commune_INSEE'].isin(INSEE_codes)]
    df_department['code_commune_INSEE']=df_department['code_commune_INSEE'].astype(int)
    df_elec['code_commune_INSEE']=df_elec['code_commune_INSEE'].astype(int)
    # Merging the two dataframes
    df_elec_department=pd.merge(df_department, df_elec, how='inner', on='code_commune_INSEE')
    df_elec_department.drop_duplicates(subset=['Consommation annuelle moyenne de la commune (MWh)', 'Nom de la commune', 'nom_departement'],inplace=True)
    df_elec_department.dropna(subset=[ 'Nom de la commune'],inplace=True)
    df_elec_department=df_elec_department[['Consommation annuelle moyenne de la commune (MWh)', 'Nom de la commune', 'nom_departement']]
    df_elec_department=df_elec_department.groupby(['nom_departement']).sum()
    return df_elec_department

# Importing data (for year 2018 only)

In [190]:
df_department=pd.read_csv("communes-departement-region.csv")
data= pd.read_csv('datamap.csv',  sep=';')

In [191]:
for year in range(2018,2022): 
    data_year=preprocess_data_per_year(data, df_department,year)
    cons[str(year)] =data_year["Consommation annuelle moyenne de la commune (MWh)"].values
df_cons_per_depart=pd.DataFrame(cons)
df_cons_per_depart["departement"]=data_year.index.values
df_cons_per_depart['avg_consumption'] = df_cons_per_depart.mean(axis=1)

In [195]:
df_cons_per_depart[['departement','avg_consumption']].to_csv("data_final.csv", index=False)

## Electricity consumption data

In [119]:
df_elec=data.loc[df_elec["Année"]==year]
print("Number of lines", df_elec["Année"].count())
df_elec.head()

Number of lines 391099


,Année,Code IRIS,Nom IRIS,Numéro de voie,Indice de répétition,Type de voie,Libellé de voie,Code INSEE de la commune,Nom de la commune,Segment de client,Nombre de logements,Consommation annuelle totale de l'adresse (MWh),Consommation annuelle moyenne par logement de l'adresse (MWh),Consommation annuelle moyenne de la commune (MWh),Adresse,Tri des adresses
22809,2018,915890103,Quartier 3,2.0,NaN,RUE,VAN GOGH,91589,SAVIGNY-SUR-ORGE,RESIDENTIEL,27,53.849,1.994,4.145,2 RUE VAN GOGH,740012
22810,2018,915890103,Quartier 3,1.0,NaN,RUE,VAN GOGH,91589,SAVIGNY-SUR-ORGE,RESIDENTIEL,56,107.966,1.928,4.145,1 RUE VAN GOGH,740013
22811,2018,627850000,Savy-Berlette (commune non irisée),19.0,NaN,RUE,DES MANOIRS,62785,SAVY-BERLETTE,RESIDENTIEL,23,149.449,6.498,6.666,19 RUE DES MANOIRS,740018
22812,2018,920710108,Musiciens-Roosevelt,11.0,NaN,AVENUE,CARNOT,92071,SCEAUX,RESIDENTIEL,10,55.877,5.588,3.788,11 AVENUE CARNOT,740047
22813,2018,920710105,Desgranges-Fontenay,2.0,NaN,SQUARE,D ALSACE,92071,SCEAUX,RESIDENTIEL,10,23.071,2.307,3.788,2 SQUARE D ALSACE,740059


In [120]:
# Remove dupplicate values for columns ...
df_elec.drop_duplicates(subset=['Code INSEE de la commune'],inplace=True)
# Drop missing values 
df_elec.dropna(subset=['Code INSEE de la commune', 'Consommation annuelle moyenne de la commune (MWh)', 'Nom de la commune'],inplace=True)

In [121]:
df_elec.count()

Année                                                            6160
Code IRIS                                                        6160
Nom IRIS                                                         6160
Numéro de voie                                                   6160
Indice de répétition                                              349
Type de voie                                                     6024
Libellé de voie                                                  6160
Code INSEE de la commune                                         6160
Nom de la commune                                                6160
Segment de client                                                6160
Nombre de logements                                              6160
Consommation annuelle totale de l'adresse (MWh)                  6160
Consommation annuelle moyenne par logement de l'adresse (MWh)    6160
Consommation annuelle moyenne de la commune (MWh)                6160
Adresse             

In [122]:
df_elec.rename(columns={'Code INSEE de la commune': 'code_commune_INSEE'}, inplace=True)
df_elec['code_commune_INSEE']=df_elec['code_commune_INSEE'].astype(str)
INSEE_codes=set(df_elec['code_commune_INSEE'])
df_elec.head()

,Année,Code IRIS,Nom IRIS,Numéro de voie,Indice de répétition,Type de voie,Libellé de voie,code_commune_INSEE,Nom de la commune,Segment de client,Nombre de logements,Consommation annuelle totale de l'adresse (MWh),Consommation annuelle moyenne par logement de l'adresse (MWh),Consommation annuelle moyenne de la commune (MWh),Adresse,Tri des adresses
22809,2018,915890103,Quartier 3,2.0,NaN,RUE,VAN GOGH,91589,SAVIGNY-SUR-ORGE,RESIDENTIEL,27,53.849,1.994,4.145,2 RUE VAN GOGH,740012
22811,2018,627850000,Savy-Berlette (commune non irisée),19.0,NaN,RUE,DES MANOIRS,62785,SAVY-BERLETTE,RESIDENTIEL,23,149.449,6.498,6.666,19 RUE DES MANOIRS,740018
22812,2018,920710108,Musiciens-Roosevelt,11.0,NaN,AVENUE,CARNOT,92071,SCEAUX,RESIDENTIEL,10,55.877,5.588,3.788,11 AVENUE CARNOT,740047
22857,2018,674450000,Scherwiller (commune non irisée),8.0,C,RUE,DE SELESTAT,67445,SCHERWILLER,RESIDENTIEL,10,13.849,1.385,5.382,8 C RUE DE SELESTAT,740320
22858,2018,742630000,Sciez (commune non irisée),524.0,NaN,AVENUE,DE BONNATRAIT,74263,SCIEZ,RESIDENTIEL,25,45.248,1.810,6.308,524 AVENUE DE BONNATRAIT,740326


## Electricity consumption per department

In [123]:
# Importing data
df_department=pd.read_csv("communes-departement-region.csv")
df_department=df_department.loc[df_department['code_commune_INSEE'].isin(INSEE_codes)]
df_department['code_commune_INSEE']=df_department['code_commune_INSEE'].astype(int)
df_elec['code_commune_INSEE']=df_elec['code_commune_INSEE'].astype(int)
df_department.head()

,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
2,1004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.960848,5.372926,4.0,NaN,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1,Ain,84.0,Auvergne-Rhône-Alpes
5,1007,AMBRONAY,1500,AMBRONAY,NaN,46.005591,5.357607,7.0,NaN,Ambronay,Ambronay,1,Ain,84.0,Auvergne-Rhône-Alpes
8,1010,ANGLEFORT,1350,ANGLEFORT,NaN,45.909372,5.795160,10.0,NaN,Anglefort,Anglefort,1,Ain,84.0,Auvergne-Rhône-Alpes
12,1014,ARBENT,1100,ARBENT,NaN,46.283494,5.690617,14.0,NaN,Arbent,Arbent,1,Ain,84.0,Auvergne-Rhône-Alpes
21,1024,ATTIGNAT,1340,ATTIGNAT,NaN,46.286180,5.179523,24.0,NaN,Attignat,Attignat,1,Ain,84.0,Auvergne-Rhône-Alpes


In [128]:
# Merging the two dataframes
df_elec_department=pd.merge(df_department, df_elec, how='inner', on='code_commune_INSEE')
df_elec_department.drop_duplicates(subset=['Consommation annuelle moyenne de la commune (MWh)', 'Nom de la commune', 'nom_departement'],inplace=True)
df_elec_department.dropna(subset=[ 'Nom de la commune'],inplace=True)

In [129]:
df_elec_department=df_elec_department[['Consommation annuelle moyenne de la commune (MWh)', 'Nom de la commune', 'nom_departement']]

In [131]:
df_elec_department.groupby(['nom_departement']).sum()

,Consommation annuelle moyenne de la commune (MWh)
nom_departement,
Ain,806.270
Aisne,354.479
Allier,148.999
Alpes-Maritimes,512.908
Alpes-de-Haute-Provence,157.925
...,...
Vendée,265.122
Vienne,88.716
Vosges,305.399


## Electricity consumption per year

In [136]:
data_2019=preprocess_data_per_year(data, df_department,2019)

In [137]:
data_2019.head(

,Consommation annuelle moyenne de la commune (MWh),Nom de la commune,nom_departement


READ the geojason data 

In [40]:
geojson_url='https://raw.githubusercontent.com/python-visualization/folium/main/examples/data/world-countries.json'
response=requests.get(geojson_url)
geojson=response.json()
##geojson

Create folium choropleth map

In [41]:
M=folium.Map(location=[20,10],zoom_start=2)
folium.Choropleth(
    geo-data=geojson,data=map_data,columns=[indicator,'Nom de la commune'],

NameError: name 'folium' is not defined